In [2]:
import numpy as np
import math

import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 
from datetime import date
import datetime

g = 9.8

In [3]:
Years = ["2000","2001","2002","2003","2004","2005","2006","2007","2008","2009"]

Months = []
for imon in range(1,13): ### shw
    Months.append(str(imon).zfill(2))
    
Days = []
for iday in range(1,30,3):
    Days.append(str(iday).zfill(2))
    
print(Years)
print(Months)
print(Days)

['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
['01', '04', '07', '10', '13', '16', '19', '22', '25', '28']


In [4]:
Nx = 24
Ny = 21
Nz = 6
N_parcel = Nx*Ny*Nz

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')
    

# directory = '../Lifetime/'
# filename = "Lifetime_"+Years[0]+Months[0]+Days[0]+".txt"
# print(directory+filename)

In [5]:
N_year  = len(Years)
N_month = len(Months)
N_day   = len(Days)

day_sink = np.zeros((Nz, Ny, Nx, N_day, N_month, N_year))

Lat_init = np.zeros((Nz, Ny, Nx, N_day, N_month, N_year))

Lon_sink = np.zeros((Nz, Ny, Nx, N_day, N_month, N_year))
Lat_sink = np.zeros((Nz, Ny, Nx, N_day, N_month, N_year))
Lev_sink = np.zeros((Nz, Ny, Nx, N_day, N_month, N_year))

Lev_sink.shape

(6, 21, 24, 10, 12, 10)

In [6]:
Pressure = [100, 75, 65, 55, 40, 30] # hPa
Height   = [16,  18, 19, 20, 22, 24] # km

In [8]:
for i_year in range(N_year):
    for i_month in tqdm(range(N_month)):
        for i_day in range(N_day):
            
            # 
            f_date = date(2000, 1, 1)
            l_date = date(int(Years[i_year]), int(Months[i_month]), int(Days[i_day]))
            Delta  = l_date - f_date
        
        
            directory = '/n/home12/hongwei/Paper_3_QBO/Lifetime_New_Tropopause/Lifetime/'
            filename = "Lifetime_"+Years[i_year]+Months[i_month]+Days[i_day]+".txt"
            File = open(directory+filename, 'r')
        
            Lines = File.readlines()

            # Strips the newline character
            data = []
            
            Lat_0 = []
            
            Longitude = []
            Latitude = []
            Pressure = []
            for line in Lines:
                a = line.split(',')
                data.append(float(a[0])) # lifetime
                
                Lat_0.append(float(a[4]))
                
                Longitude.append(float(a[6]))
                Latitude.append(float(a[7]))
                Pressure.append(float(a[8]))
    
    
            i = 0
            for ix in range(Nx):
                for iy in range(Ny):
                    for iz in range(Nz):
                        day_sink[iz,iy,ix,i_day,i_month,i_year] = data[i] + Delta.days  
                        
                        Lat_init[iz,iy,ix,i_day,i_month,i_year] = Lat_0[i]
                        
                        Lon_sink[iz,iy,ix,i_day,i_month,i_year] = Longitude[i]
                        Lat_sink[iz,iy,ix,i_day,i_month,i_year] = Latitude[i]
                        Lev_sink[iz,iy,ix,i_day,i_month,i_year] = Pressure[i]

                        i = i+1

100%|██████████| 12/12 [00:10<00:00,  1.11it/s]


In [9]:
Lats_edge = np.arange(-90,91,6)
Lons_edge = np.arange(-180,181,10)
Levs_edge = np.arange(75,450,50)

Lats_mid = np.arange(-87,90,6)
Lons_mid = np.arange(-175,180,10)
Levs_mid = np.arange(100,450,50)


N_lat = len(Lats_mid)
N_lon = len(Lons_mid)
N_lev = len(Levs_mid) # 16, 18, 20, 22, 24 km

Lats_edge, Lons_edge, Levs_edge, Lats_mid, Lons_mid, Levs_mid, N_lat, N_lon, Lons_edge[-1]

(array([-90, -84, -78, -72, -66, -60, -54, -48, -42, -36, -30, -24, -18,
        -12,  -6,   0,   6,  12,  18,  24,  30,  36,  42,  48,  54,  60,
         66,  72,  78,  84,  90]),
 array([-180, -170, -160, -150, -140, -130, -120, -110, -100,  -90,  -80,
         -70,  -60,  -50,  -40,  -30,  -20,  -10,    0,   10,   20,   30,
          40,   50,   60,   70,   80,   90,  100,  110,  120,  130,  140,
         150,  160,  170,  180]),
 array([ 75, 125, 175, 225, 275, 325, 375, 425]),
 array([-87, -81, -75, -69, -63, -57, -51, -45, -39, -33, -27, -21, -15,
         -9,  -3,   3,   9,  15,  21,  27,  33,  39,  45,  51,  57,  63,
         69,  75,  81,  87]),
 array([-175, -165, -155, -145, -135, -125, -115, -105,  -95,  -85,  -75,
         -65,  -55,  -45,  -35,  -25,  -15,   -5,    5,   15,   25,   35,
          45,   55,   65,   75,   85,   95,  105,  115,  125,  135,  145,
         155,  165,  175]),
 array([100, 150, 200, 250, 300, 350, 400]),
 30,
 36,
 180)

In [10]:
def find_i_LON(lon):
    if lon<Lons_edge[0]:  lon = lon+360
    if lon>=Lons_edge[-1]: lon = lon-360
    return int( np.floor( (lon - Lons_edge[0]) / (Lons_edge[1]-Lons_edge[0]) ) )


def find_i_LAT(lat):
    return int( np.floor( (lat - Lats_edge[0]) / (Lats_edge[1]-Lats_edge[0]) ) )


def find_i_LEV(lev):
    if lev<Levs_edge[0]:  lev = Levs_mid[0]
    if lev>=Levs_edge[-1]: lev = Levs_mid[-1]
    return int( np.floor( (lev - Levs_edge[0]) / (Levs_edge[1]-Levs_edge[0]) ) )

In [11]:
Levs_edge, Levs_edge[N_lev]

(array([ 75, 125, 175, 225, 275, 325, 375, 425]), 425)

In [12]:
Total_Mon = 20*12
Sink_num = np.zeros((N_lon, N_lat, N_lev, Total_Mon))


In [13]:
# Lon_sink = np.zeros((Nz, Ny, Nx, N_day, N_month, N_year))

for iyear in tqdm(range(N_year)):
    for imon in range(N_month):
        for iday in range(N_day):
            for ix in range(Nx):
                for iy in range(Ny):
                    iz=2
                    
                    if abs(Lat_init[iz, iy, ix, iday, imon, iyear])<=15.0:
                    
                        i_LON = find_i_LON(Lon_sink[iz, iy, ix, iday, imon, iyear])
                        i_LAT = find_i_LAT(Lat_sink[iz, iy, ix, iday, imon, iyear])
                        i_LEV = find_i_LEV(Lev_sink[iz, iy, ix, iday, imon, iyear])
                    
                        start_date = datetime.datetime.strptime("01/01/00", "%m/%d/%y")
                        end_date = start_date + datetime.timedelta(days=day_sink[iz, iy, ix, iday, imon, iyear])
                        i_MON = 12*(end_date.year-2000) +end_date.month -1
                          
#                     print(LON, LAT, LEV, i_LON, i_LAT, i_LEV, it)
                        Sink_num[i_LON, i_LAT, i_LEV, i_MON] = Sink_num[i_LON, i_LAT, i_LEV, i_MON] + 1

100%|██████████| 10/10 [00:06<00:00,  1.51it/s]


In [14]:
i_LON, i_LAT, i_LEV, i_MON, Sink_num[i_LON, i_LAT, i_LEV, i_MON]

(6, 23, 4, 158, 1.0)

### The unit is: particles per month

In [15]:
Height[:]

[16, 18, 19, 20, 22, 24]

In [16]:
# Sink_num = np.zeros((N_lon, N_lat, N_lev, Total_Mon))

with open(f'./Num_Sinks_Within_15deg_{Height[iz]:.0f}km.txt', 'w') as f:
    for i_MON in range(Total_Mon):
            for ix in range(N_lon):
                for iy in range(N_lat):
                    f.write(  str(i_MON) + ',' \
                            + str(Lons_mid[ix]) + ',' \
                            + str(Lats_mid[iy]) + ',' \
                        
                            + str(Sink_num[ix, iy, 0, i_MON])  + ',' \
                            + str(Sink_num[ix, iy, 1, i_MON])  + ',' \
                            + str(Sink_num[ix, iy, 2, i_MON])  + ',' \
                            + str(Sink_num[ix, iy, 3, i_MON])  + ',' \
                            + str(Sink_num[ix, iy, 4, i_MON])  + ',' \
                            + str(Sink_num[ix, iy, 5, i_MON])  + ',' \
                            + str(Sink_num[ix, iy, 6, i_MON])   )
                        
                    f.write('\n')
f.close()

In [17]:
# Check whether the total number of sinks are same as the total number of injected particles

np.sum(Sink_num)/(10*(360/3)*Nx*Ny), Nx*Ny*Nz

(0.5238095238095238, 3024)

In [18]:
###
# fig = plt.figure(figsize=(12,10), dpi=150)
# projection = crs.PlateCarree()

# ax1 = fig.add_subplot(4,1,1, projection=projection)

# ax1.set_global()
# # ax1.set_extent(extent, crs=projection) ## Important
# ax1.add_feature(cfeature.COASTLINE, edgecolor="gray")

# ax1_pixel = plt.imshow(lifetime_season_mean_new2[iz,:,:,0]/365.25, vmin=Day_min, vmax=Day_max,\
#            extent = extent, cmap='RdBu_r', interpolation='nearest')

# cb1 = plt.colorbar(ax1_pixel, ax=ax1)
# cb1.set_label('[year]', fontsize=12);

# ax1.set_xticks(np.linspace(-180, 180, 13), crs=projection)
# ax1.set_yticks(np.linspace(-30, 30, 5), crs=projection)

# ax1.set_title(f"Max: {int(np.max(lifetime_season_mean_new2[iz,:,:,0]))/365.25:.1f} years", loc='right', fontsize=12);
# ax1.set_title(f"Min: {int(np.min(lifetime_season_mean_new2[iz,:,:,0]))/365.25:.1f} years", loc='left', fontsize=12);
# ax1.set_title(f"Lifetime of particles (injected at {np.round(Height[iz])} km) in Winter")



In [19]:
import datetime

date_1 = datetime.datetime.strptime('01/01/00', "%m/%d/%y")

end_date = date_1 + datetime.timedelta(days=10)

end_date

datetime.datetime(2000, 1, 11, 0, 0)